## 결정 트리, 랜덤 포레스트, 에이다부스트. 
#### 필요한 패키지를 불러온다:

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
from scipy.stats import itemfreq

#### 데이터를 불러온다:

In [3]:
os.chdir(r"D:\1. stark\anaconda_workspace\no.2\머신러닝 알고리즘과 응용\data")

In [4]:
# df = pd.read_csv('data_spam.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_spam.csv', header='infer',encoding='latin1')
df

In [5]:
X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns

'yes', 'no' 레이블을 숫자로 변환:

In [6]:
LE = preprocessing.LabelEncoder()
Y = LE.fit_transform(Y)

In [7]:
table = itemfreq(Y)
table

D:\Users\bit\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[  0, 736],
       [  1, 264]], dtype=int64)

NaN이 있으면 채워 넣음:

In [8]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

array([[3.227e+03, 1.060e+00, 2.600e-01, ..., 0.000e+00, 0.000e+00,
        3.900e-02],
       [2.331e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        2.200e-01],
       [1.899e+03, 4.200e-01, 0.000e+00, ..., 0.000e+00, 4.200e-01,
        9.700e-02],
       ...,
       [2.433e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        4.780e-01],
       [4.840e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.280e+00,
        6.320e-01],
       [2.636e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

#### 데이터 전처리:

In [8]:
X = preprocessing.scale(X)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)

#### Tree 적용 (교차검증 최적화 포함):

In [10]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}

In [11]:
gridCV = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']

In [12]:
print("Tree best depth : " + str(best_depth))

Tree best depth : 28


In [13]:
DTC_best = DecisionTreeClassifier(max_depth=best_depth)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Tree best accuracy : 0.872


#### Random Forest 적용 (교차검증 최적화 포함):

Tree 하고는 스텝에서 조금 차이가 있음:

In [14]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']

In [15]:
print("Random Forest best n estimator : " + str(best_n_estim))
print("Random Forest best depth : " + str(best_depth))

Random Forest best n estimator : 21
Random Forest best depth : 7


In [16]:
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(X_train, Y_train)
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Random Forest best accuracy : 0.928


#### AdaBoost 적용 (교차검증 최적화 포함): <br>
디폴트: base_estimator=DecisionTreeClassifier 

In [17]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']

In [18]:
print("Ada Boost best n estimator : " + str(best_n_estim))
print("Ada Boost best learning rate : " + str(best_learn_rate))

Ada Boost best n estimator : 40
Ada Boost best learning rate : 0.5


In [19]:
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(X_train, Y_train);
Y_pred = AB_best.predict(X_test)
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Ada Boost best accuracy : 0.934
